# ML Model 1

### Primary Type

How can various environmental, temporal, and community-related factors be used to predict the primary type of crime in a given location, and what insights does this provide into the patterns of criminal activity in urban environments like Chicago?

In [2]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1716473697126_0002,pyspark,idle,Link,Link,
1,application_1716473697126_0003,pyspark,idle,Link,Link,
2,application_1716473697126_0004,pyspark,idle,Link,Link,


In [3]:
sc.install_pypi_package("matplotlib==3.2.1", "https://pypi.org/simple")
sc.install_pypi_package("pandas==1.0.5", "https://pypi.org/simple")
sc.install_pypi_package("scipy==1.4.1", "https://pypi.org/simple")
sc.install_pypi_package("seaborn==0.11.2", "https://pypi.org/simple")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1716473697126_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b2/c2/71fcf957710f3ba1f09088b35776a799ba7dd95f7c2b195ec800933b276b/matplotlib-3.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9d/ea/6d76df31432a0e6fdf81681a895f009a4bb47b3c39036db3e1b528191d52/pyparsing-3.1.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f9/77/e3046bf19720b22e3e0b7c12e28f6f2c0c18a213fb91a56cea640862270f/kiwisolver-1.4.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ec/6b/63cc3df74987c36fe26157ee12e09e8f9db4de771e0f3404263117e75b95/typing_extensions-4.7.1-py3-none-a

In [50]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import year, month
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, Bucketizer,StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
crimes = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace", True) \
    .option("header", "true") \
    .parquet("s3://hvpachisia-chicago-crime/processed_data/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
crimes.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- community_area: string (nullable = true)
 |-- id: string (nullable = true)
 |-- case_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- block: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- year: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- location: string (nullable = true)
 |-- month: string (nullable = true)
 |-- community_name: string (nullable = true)
 |-- birth_rate: string (nullable = true)
 |-- below_poverty_level: double (nullable = true)
 |-- crowded_housing: double (nullable = true)
 |

In [72]:
all_columns = crimes.schema.names

columns_to_drop = [col for col in all_columns if "index" in col or "vec" in col]

crimes = crimes.drop(*columns_to_drop)

crimes.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- community_area: string (nullable = true)
 |-- id: string (nullable = true)
 |-- case_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- block: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- year: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- location: string (nullable = true)
 |-- month: string (nullable = true)
 |-- community_name: string (nullable = true)
 |-- birth_rate: string (nullable = true)
 |-- below_poverty_level: double (nullable = true)
 |-- crowded_housing: double (nullable = true)
 |

In [89]:
boolean_columns = ["arrest", "domestic"]
for column in boolean_columns:
    crimes = crimes.withColumn(column, F.when(col(column) == "true", 1).otherwise(0))

categorical_columns = ["community_area", "description", "district", "ward", "location_description", "season"]
numerical_columns = ["latitude", "longitude", "below_poverty_level", "crowded_housing", 
                     "no_high_school_diploma", "per_capita_income", "unemployment"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
# Indexing and encoding categorical columns
indexers = [StringIndexer(inputCol=c, outputCol=c+"_index", handleInvalid="keep") for c in categorical_columns]
encoders = [OneHotEncoder(inputCol=c+"_index", outputCol=c+"_vec") for c in categorical_columns]

# indexing the target variable
primary_type_indexer = StringIndexer(inputCol="primary_type", outputCol="primary_type_index", handleInvalid="keep")

# Assemble features into a single vector column
assembler_inputs = [c+"_vec" for c in categorical_columns] + numerical_columns
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features", handleInvalid="skip")

# Standardize the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
train_full, test = crimes.randomSplit([0.8, 0.2], seed=0)
train_subset = train_full.sample(fraction=0.001, seed=0)  
test.persist()
train_subset.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[community_area: string, id: string, case_number: string, date: string, block: string, primary_type: string, description: string, location_description: string, arrest: int, domestic: int, beat: string, district: string, ward: string, x_coordinate: string, y_coordinate: string, year: string, latitude: double, longitude: double, location: string, month: string, community_name: string, birth_rate: string, below_poverty_level: double, crowded_housing: double, no_high_school_diploma: double, per_capita_income: int, unemployment: double, hourofday: int, dayofweek: int, season: string, isweekend: int, areacrimecount: bigint, timesincelastcrime: bigint, prevyearcrimecount: bigint, yearlycrimeratechange: double, nextcrimedate: string, recurrentcrime: int, PrevCrimesAtLocation: bigint]

In [92]:
classifier = RandomForestClassifier(labelCol="primary_type_index", featuresCol="scaledFeatures")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
pipeline = Pipeline(stages= [primary_type_indexer]+ indexers + encoders + [assembler, scaler, classifier])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Define the parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(classifier.numTrees, [10, 20, 50]) \
    .addGrid(classifier.maxDepth, [5, 10, 20]) \
    .build()

evaluator = MulticlassClassificationEvaluator(
    labelCol="primary_type_index", 
    predictionCol="prediction", 
    metricName="accuracy")

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
# Train the model
model = crossval.fit(train_subset)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
# Make predictions
predictions = model.transform(test)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="primary_type_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.2f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy: 0.82

In [100]:

predictionAndLabels = predictions.select("prediction", "primary_type_index")\
    .rdd.map(lambda x: (float(x[0]), float(x[1])))

metrics = MulticlassMetrics(predictionAndLabels)

labels = predictions.select("primary_type_index").distinct().rdd.flatMap(lambda x: x).collect()
for label in sorted(labels):
    precision = metrics.precision(label)
    recall = metrics.recall(label)
    f1 = metrics.fMeasure(label)

weighted_precision = metrics.weightedPrecision
weighted_recall = metrics.weightedRecall
weighted_f1 = metrics.weightedFMeasure()

print(f"Weighted Precision: {weighted_precision:.2f}")
print(f"Weighted Recall: {weighted_recall:.2f}")
print(f"Weighted F1 Score: {weighted_f1:.2f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Weighted Precision: 0.82
Weighted Recall: 0.82
Weighted F1 Score: 0.81
Confusion Matrix:
[[3.09387e+05 2.04600e+03 2.00000e+00 1.22600e+03 0.00000e+00 0.00000e+00
  2.20000e+01 0.00000e+00 0.00000e+00 2.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00]
 [8.45100e+03 2.30095e+05 3.00000e+00 4.15700e+03 2.75580e+04 0.00000e+00
  7.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00]
 [3.30500e+03 1.27200e+03 1.63389e+05 3.97000e+02 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.000

The Random Forest classifier achieved a weighted precision and recall of 82%, and a weighted F1 score of 81% in predicting the primary type of crime. These metrics indicate that the model is generally effective at correctly identifying the type of crime while balancing precision and recall across different crime types. Precision measures the proportion of true positive predictions among all positive predictions, while recall measures the proportion of true positives out of the actual positives. The most significant features include below_poverty_level, community_area_vec, and description_vec, each contributing more substantially than other features.

Overall, the model's high accuracy and balanced precision and recall suggest it is a reliable tool for classifying crime types based on the available features. Understanding the importance of different features can guide further improvements in crime prevention strategies and resource allocation, emphasizing the role of socioeconomic conditions and geographic location in crime dynamics.